# Model comparison

In [80]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklego.metrics import equal_opportunity_score
from sklego.metrics import p_percent_score
from sklearn.metrics import log_loss
from sklearn.utils.extmath import squared_norm
from moopt.scalarization_interface import scalar_interface, single_interface, w_interface
from moopt import monise
import numpy as np
import optuna, sklearn, sklearn.datasets
from fair_models import coefficient_of_variation, MOOLogisticRegression, FindCLogisticRegression, FindCCLogisticRegression
from fair_models import calc_reweight
from fair_models import FairScalarization, EqualScalarization

In [83]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


## Parte 1: Data treatment

In [84]:
mydata= pd.read_csv("Datasets/german_credit_data.csv")

Dados de pedidos de crédito. É um dos datasets mais utilizados para tutoriais em Fairness, como na biblioteca [$aif360$](https://github.com/IBM/AIF360/blob/master/examples/README.md). Dataset original disponível em [aqui](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)), mas eu utilizei [este](https://www.kaggle.com/kabure/german-credit-data-with-risk), por estar em formato csv com os headers, embora omita informações do dataset original.

Originalmente possui 1000 dados, mas com vários valores NaN, ficando com 522 dados após remoção de dados com informações faltantes.

In [85]:
mydata.head()

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


In [86]:
mydata = mydata.drop(['Unnamed: 0', 'Purpose'], axis=1)

In [87]:
mydata = mydata.dropna()

In [88]:
mapping_Sex = {'male': 0, 'female': 1}
mapping_Housing = {'free': 1, 'rent': 2, 'own': 3}
mapping_Savings = {'little': 1, 'moderate': 2, 'quite rich': 3, 'rich': 4}
mapping_Checking = {'little': 1, 'moderate': 2, 'rich': 3}
mapping_Risk = {"bad": -1, "good": 1}

numerical_data = mydata.replace({'Sex': mapping_Sex, 'Housing': mapping_Housing, 'Saving accounts': mapping_Savings,
                'Checking account':mapping_Checking, 'Risk': mapping_Risk})

In [89]:
X = numerical_data.drop(['Risk'], axis=1)

In [90]:
y = numerical_data['Risk']

In [91]:
# 395, 808
random_seed = 2000#np.random.randint(0, 1000)
random_seed

2000

In [92]:
random_seed2 = 2000#np.random.randint(0, 1000)
random_seed2

2000

In [93]:
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=200, random_state = random_seed)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=100, random_state = random_seed2)

In [94]:
sample_weight = calc_reweight(X_train, y_train)

## Parte 3: Comparando os modelos

In [95]:
optimize_metrics = {'accuracy': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []},
                'equal_opportunity': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []},
                'p_percent': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []},
                'c_variation': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []}}

In [96]:
metrics = ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation']

In [97]:
methods = ['LogReg', 'RegEqual', 'RegDemo', 'RegRewe', 'RegMoo', 'RegEqMoo']
index = [sum([[method]*len(metrics) for method in methods],[]),
        sum([metrics for method in methods],[])]
results_test = pd.DataFrame(index=index, columns=metrics)
results_val = pd.DataFrame(index=index, columns=metrics)

In [98]:
# RegLog

for metric in metrics:
    reg_log = FindCLogisticRegression(X_train, y_train, X_val, y_val, metric=metric).tune()
    # RegMoo

    optimize_metrics[metric]['LogReg'] = [reg_log.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_log, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_log, X_test),
                            coefficient_of_variation(reg_log, X_test, y_test)]
    results_test.loc[('LogReg',metric),:] = [reg_log.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_log, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_log, X_test),
                            coefficient_of_variation(reg_log, X_test, y_test)]

In [99]:
# RegEqual

for metric in metrics:
    reg_equal = FindCCLogisticRegression(X_train, y_train, X_val, y_val, metric=metric, base_model='equal').tune()

    optimize_metrics[metric]['RegEqual'] = [reg_equal.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_equal, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_log, X_test),
                            coefficient_of_variation(reg_equal, X_test, y_test)]
    results_test.loc[('RegEqual',metric),:] = [reg_equal.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_equal, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_log, X_test),
                            coefficient_of_variation(reg_equal, X_test, y_test)]

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

In [100]:
# RegDemo

for metric in metrics:
    reg_demo = FindCCLogisticRegression(X_train, y_train, X_val, y_val, metric=metric, base_model='demographic').tune()

    optimize_metrics[metric]['RegDemo'] = [reg_demo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_demo, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_demo, X_test),
                            coefficient_of_variation(reg_demo, X_test, y_test)]
    results_test.loc[('RegDemo',metric),:] = [reg_demo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_demo, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_demo, X_test),
                            coefficient_of_variation(reg_demo, X_test, y_test)]

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

In [101]:
# RegRewe

for metric in metrics:
    reg_rewe = FindCLogisticRegression(X_train, y_train, X_val, y_val, metric=metric, sample_weight=sample_weight).tune()

    optimize_metrics[metric]['RegRewe'] = [reg_rewe.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_rewe, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_rewe, X_test),
                            coefficient_of_variation(reg_rewe, X_test, y_test)]
    results_test.loc[('RegRewe',metric),:] = [reg_rewe.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_rewe, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_rewe, X_test),
                            coefficient_of_variation(reg_rewe, X_test, y_test)]

In [102]:
# RegMoo

scalarization = FairScalarization(X_train, y_train, 'Sex')
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, scalarization)
moo_learn.tune()

for metric in metrics:
    reg_moo = moo_learn.tune(metric)
    
    results_test.loc[('RegMoo',metric),:] = [reg_moo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_moo, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_moo, X_test),
                            coefficient_of_variation(reg_moo, X_test, y_test)]

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWar

In [103]:
# RegEqMoo

scalarization = EqualScalarization(X_train, y_train, 'Sex')
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, scalarization)
moo_learn.tune()

for metric in metrics:
    reg_eqmoo = moo_learn.tune(metric)
    
    results_test.loc[('RegEqMoo',metric),:] = [reg_eqmoo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_eqmoo, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_eqmoo, X_test),
                            coefficient_of_variation(reg_eqmoo, X_test, y_test)]

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWar

In [104]:
results_test

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy             0.615          0.903195   0.96966    0.677477
         equal_opportunity    0.535                 1         1    0.481483
         p_percent            0.535                 1         1    0.481483
         c_variation          0.535                 1         1    0.481483
RegEqual accuracy              0.59          0.951754         1    0.572637
         equal_opportunity      0.6          0.932331         1    0.657267
         p_percent             0.61          0.956984         1    0.706982
         c_variation           0.56                 1         1    0.487498
RegDemo  accuracy             0.585          0.938158  0.894118      0.5829
         equal_opportunity    0.535                 1         1    0.481483
         p_percent            0.535                 1         1    0.481483
         c_variation          0.535                 1         1    0.481483
RegRewe  accuracy              0.59          0.951754  0.925224    0.572637
         equal_opportunity    0.535                 1         1    0.481483
         p_percent            0.535                 1         1    0.481483
         c_variation          0.535                 1         1    0.481483
RegMoo   accuracy              0.62          0.897368  0.958935    0.617094
         equal_opportunity    0.585          0.941296  0.980392     0.71675
         p_percent             0.59          0.909919  0.931373     0.73646
         c_variation           0.55          0.967742  0.990602    0.497292
RegEqMoo accuracy              0.63          0.842982  0.990291    0.657375
         equal_opportunity    0.535                 1         1    0.481483
         p_percent            0.535                 1         1    0.481483
         c_variation          0.535                 1         1    0.481483

In [25]:
def dominate(a, b):
    sense = np.array([1, 1, 1, -1])
    if all((sense*a)>=(sense*b)) and any((sense*a)>(sense*b)):
        return True
    else:
        return False

In [26]:
new_results = results_test.copy()//0.01/100
dominate_metr = [any([dominate(other, row) for other in new_results.values]) for row in new_results.values]
new_results['dominated'] = dominate_metr
new_results

accuracy equal_opportunity p_percent c_variation  \
LogReg   accuracy              0.61               0.9      0.96        0.67   
         equal_opportunity     0.56              0.99      0.98        0.48   
         p_percent             0.53              0.99      0.99        0.48   
         c_variation           0.53              0.99      0.99        0.48   
RegEqual accuracy              0.58              0.95      0.99        0.57   
         equal_opportunity     0.54              0.99      0.99        0.48   
         p_percent             0.61              0.95      0.99         0.7   
         c_variation           0.53              0.99      0.99        0.48   
RegDemo  accuracy              0.58              0.95      0.92        0.57   
         equal_opportunity     0.53              0.99      0.99        0.48   
         p_percent             0.61              0.94      0.92        0.71   
         c_variation           0.53              0.99      0.99        0.48   
RegRewe  accuracy              0.58              0.95      0.92        0.57   
         equal_opportunity     0.53              0.99      0.99        0.48   
         p_percent             0.53              0.99      0.99        0.48   
         c_variation           0.53              0.99      0.99        0.48   
RegMoo   accuracy              0.59              0.96      0.93        0.56   
         equal_opportunity     0.59              0.96      0.93        0.56   
         p_percent             0.56              0.98      0.95        0.77   
         c_variation           0.58              0.98      0.98        0.51   

                            dominated  
LogReg   accuracy               False  
         equal_opportunity      False  
         p_percent               True  
         c_variation             True  
RegEqual accuracy               False  
         equal_opportunity      False  
         p_percent              False  
         c_variation             True  
RegDemo  accuracy                True  
         equal_opportunity       True  
         p_percent               True  
         c_variation             True  
RegRewe  accuracy                True  
         equal_opportunity       True  
         p_percent               True  
         c_variation             True  
RegMoo   accuracy               False  
         equal_opportunity      False  
         p_percent               True  
         c_variation            False

In [27]:
print(new_results.to_latex())

\begin{tabular}{lllllll}
\toprule
       &             & accuracy & equal\_opportunity & p\_percent & c\_variation &  dominated \\
\midrule
LogReg & accuracy &     0.61 &               0.9 &      0.96 &        0.67 &      False \\
       & equal\_opportunity &     0.56 &              0.99 &      0.98 &        0.48 &      False \\
       & p\_percent &     0.53 &              0.99 &      0.99 &        0.48 &       True \\
       & c\_variation &     0.53 &              0.99 &      0.99 &        0.48 &       True \\
RegEqual & accuracy &     0.58 &              0.95 &      0.99 &        0.57 &      False \\
       & equal\_opportunity &     0.54 &              0.99 &      0.99 &        0.48 &      False \\
       & p\_percent &     0.61 &              0.95 &      0.99 &         0.7 &      False \\
       & c\_variation &     0.53 &              0.99 &      0.99 &        0.48 &       True \\
RegDemo & accuracy &     0.58 &              0.95 &      0.92 &        0.57 &       True \\
      

## Models with ensemble

In [66]:
ensemble_metrics = {'accuracy': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []},
                'equal_opportunity': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []},
                'p_percent': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []},
                'c_variation': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []}}

In [67]:
ensembles = ['voting soft', 'voting hard', 'knorau', 'knorae']

In [68]:
methods = ['RegMoo', 'RegEqMoo']
index = [sum([[method]*len(ensembles) for method in methods],[]),
        sum([ensembles for method in methods],[])]
results_test = pd.DataFrame(index=index, columns=metrics)

In [81]:
# RegMoo

scalarization = FairScalarization(X_train, y_train, 'Sex')
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, scalarization)

for ensemble in ensembles:
    reg_moo = moo_learn.ensemble_model(ensemble)
    
    results_test.loc[('RegMoo',ensemble),:] = [reg_moo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_moo, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_moo, X_test),
                            coefficient_of_variation(reg_moo, X_test, y_test)]

In [77]:
# RegEqMoo

scalarization = EqualScalarization(X_train, y_train, 'Sex')
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, scalarization)

for ensemble in ensembles:
    reg_eqmoo = moo_learn.ensemble_model(ensemble)
    
    results_test.loc[('RegEqMoo',ensemble),:] = [reg_eqmoo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_eqmoo, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_eqmoo, X_test),
                            coefficient_of_variation(reg_eqmoo, X_test, y_test)]

In [82]:
results_test

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft    0.605           0.98836  0.939904    0.687403
         voting hard    0.605          0.936647         1    0.687403
         knorau         0.605          0.936647         1    0.687403
         knorae         0.605          0.913684  0.930719    0.764822
RegEqMoo voting soft    0.535                 1         1    0.481483
         voting hard    0.535                 1         1    0.481483
         knorau         0.565          0.993897  0.979749    0.523966
         knorae           0.6          0.929922  0.951872    0.774627